# GGUF Format

GGUF is the current state of the art model, started being supported for llama.cpp on August 23rd, 2023. Today is August 25th, so pretty recent, you could say! GGML is its predacessor and has since been depricated.

To use, I recommend using ctransformers, but you can also use the llama.cpp library directly as well. Be sure that you have the most updated versions, ctransformers required version >=0.2.24 for gguf support, and llama.cpp python bindings does not yet support it -- you need to be careful

In [1]:
from ctransformers import AutoModelForCausalLM
import textwrap

llm = AutoModelForCausalLM.from_pretrained(
    model_path_or_repo_id="msaad02/llama2_7b_brockportgpt_gguf",
    model_file="brockportgpt-7b-q4_1.gguf",
    model_type="llama"
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
llm_config = {
    "max_new_tokens": 256,
    "stream": True,
}

def qa(text: str, full = False):
    # textwrap.dedent gets rid of indenting at the start of each newline
    text = textwrap.dedent(f"""\
        Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

        ### Instruction:
        {text}

        ### Response:
        """)
    
    generator = llm(text, **llm_config)

    # print beginning?
    if full:
        print(text, end="")

    response = ""
    for token in generator:
        response = response + token
        print(token, end="")

    response = (text + response) if full else response

    return response

In [3]:
output = qa("How do I apply?", full=True)

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
How do I apply?

### Response:
You can apply online using the common app on our website.

In [4]:
output = qa("is there a CS department?")

Yes, there is a Department of Computer Science in the School of Arts & Sciences at SUNY Brockport.

In [5]:
output = qa("Tell me about the CS department at Brockport.")

# yikes!! Didn't follow the format perfectly, and kept going. No good.

The CS department at Brockport is dedicated to providing students with a comprehensive education in computer science, allowing them to develop their skills and gain hands-on experience in various fields.

In [6]:
def sample_qa(text: str, full = False, n = 3):
    """
    For a given question, run the model n times to see a variety of responses.
    """
    answer_list = []

    print("1. ", end="")
    for num in range(n):
        answer_list.append(qa(text, full))

        if num+1 < n:
            print(f"\n\n{num+2}. ", end="")

    return answer_list

In [7]:
output = sample_qa("Tell me about the CS department at Brockport.")

1. The Computer Science (CS) Department offers a variety of courses and programs in computer science, including a major, minor, and certificate.

2. The Computer Science (CS) Department provides undergraduate and graduate programs in computer science, information systems, cybersecurity, information assurance, web development, and more.

3. The CS department at Brockport is committed to providing a strong foundation in computing, offering both traditional and emerging technologies such as blockchain, cybersecurity, and data science.

In [8]:
output = sample_qa("Im an undergrad. What are the steps to apply?")

1. The steps to apply as an undergraduate at SUNY Brockport include submitting your application, sending in any required documents (e.g., transcripts), and paying the application fee if applicable.

2. The steps to apply as an undergraduate for graduate study at SUNY Brockport include submitting a non-degree application, completing two foundation courses if not transferring in any credit, and earning a 3.0 GPA or higher.

3. To apply as an undergraduate at SUNY Brockport, you will need to complete the online application and pay the $50 application fee.